In [11]:

import pandas as pd
import numpy as np
import pickle as pkl
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np

import time
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sb
# import visual tools
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')

#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
#plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
#plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

# import util libs

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 777
###
import pystan
from scipy.stats import pearsonr, zscore
from MFDFA import fgn
# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sb.set()
from tqdm import tqdm

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import minmax_scale
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import QuantileTransformer
# from sklearn.preprocessing import PowerTransformer

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')

import stylised_facts_data_utilities.createLOB as createLOB

%matplotlib inline
import os
os.getcwd()


'/home/ak/Documents/Research/PaperCode/stylised_facts'

In [12]:
folder= '/media/ak/My Passport/Experiment Data/ActivityClockData/'
folderList = os.listdir(folder)
folderList
clocksDataList =[s for s in folderList if ('ClocksData') in s]
dddddd

In [13]:
idxFile = 20
fileLoc = "".join((folder, clocksDataList[idxFile]))
symbolID =fileLoc.split("_")[1:3]
symbolText=("".join((symbolID[0]," ", symbolID[1])))

In [14]:
fileLoc

'/media/ak/My Passport/Experiment Data/ActivityClockData/ClocksData_G_1_Comdty_20180504_.pkl'

In [13]:

pickle_to_file = pkl.load(open(fileLoc, "rb"))

In [14]:
keys = pickle_to_file.keys()

In [15]:
keys

dict_keys(['OriginalDF', 'TickBarDf', 'VolumeBarDf', 'DollarVolumeBarDf'])

In [16]:
originaldf=pickle_to_file[list(keys)[0]]
tickdf=pickle_to_file[list(keys)[1]]
volumedf = pickle_to_file[list(keys)[2]]
dollarvolumedf = pickle_to_file[list(keys)[3]]

In [17]:
dollarvolumedf.columns.values

array(['BidQuoteId', 'BidSize', 'QuoteTime', 'type_x', 'BestBid',
       'TradeTime', 'AskQuoteId', 'AskSize', 'time_y', 'type_y',
       'BestAsk', 'TradeId', 'TradeVolume', 'TradedTime', 'type',
       'TradePrice', 'TradeId', 'TimeStamp', 'milliSeconds',
       'DollarVolume', 'MicroPrice', 'TradeSize', 'DollarVolumeTraded'],
      dtype=object)

In [18]:

testY =np.array(volumedf.MicroPrice.pct_change().dropna())

In [19]:
def returnsPulse(x):
    cumsum = np.cumsum(x)
    y =  np.vstack([cumsum, np.triu( np.expand_dims( cumsum, axis=0 ) - np.outer( np.ones( len( cumsum )  ), cumsum ).transpose() ) ] ) 
    return y

In [20]:
# def returnsPulse(x):
#     cumsum = np.cumsum(x)
#     y = np.triu(np.vstack( [cumsum] + [cumsum - cumsum[i] for i in range(len(cumsum))]))
#     return pd.DataFrame(y)

In [21]:
y_df =pd.DataFrame(returnsPulse(testY))

In [ ]:
def rollingSum(x):
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x) + 1, len( x )])
    result[0, :] =cumsum
    for i in range( len( x )):
        result[ i+1, :] = (cumsum - cumsum[i])
    return np.triu( result )
        

    
def rollingMean(x):
    """
    Compute rolling mean of each bucket shifted by on time in each row
    """
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x + 1), len( x )])
    den = np.arange(1, len( x ) + 1)
    result[0, :] =cumsum/den
    for i in range(len( x)):
        result[ i+1, :] = (cumsum - cumsum[i])/(den - (i +1))
    return np.triu( result)
    
 

In [7]:
def return_tstats_over_days( xs):
    """
    computes the t-statistic over multiple windows across many days
    
    """
    T, N = xs.shape
    result = np.zeros( [N+1, N])
    # compute the cumulative returns for trades with entry at tick at n=0
    cumsum = np.cumsum(x, axis =1) #of shape [T, N]
    
    result[0, :] = np.mean( cumsum, axis=0)/np.std( cumsum, axis=0)
    for n in range(N):
        ##subtract cumsum of shape [T,N] with cumsum[:,n] of [T,1], to compute the returns for trades with entry at n-th tick
        returns = cumsum - np.expand_dims(cumsum[:, n], axis =1)
        ##then compute the t-statistics over time(first) dimension
        result[n+1, :]= np.mean(returns, axis=0)/np.std(returns, axis=0)
    return np.triu(result)

In [5]:
x = np.random.normal(size =[10,20000])

In [8]:
t_stats= return_tstats_over_days(x)

In [10]:
pd.DataFrame(t_stats)

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,0.273789,-0.024447,0.003448,-0.029653,0.001664,-0.013246,0.051859,0.036808,-0.033215,-0.048901,...,0.572963,0.574414,0.575873,0.571847,0.574758,0.577242,0.578594,0.581253,0.582167,0.581713
1,0.000000,-0.231299,-0.134008,-0.127701,-0.094280,-0.128334,-0.024857,-0.045806,-0.107380,-0.134557,...,0.571871,0.573317,0.574772,0.570746,0.573650,0.576131,0.577484,0.580149,0.581064,0.580612
2,0.000000,0.000000,0.055343,-0.023914,0.026455,0.007142,0.079358,0.062743,-0.025075,-0.040353,...,0.574745,0.576203,0.577650,0.573602,0.576505,0.578985,0.580347,0.583018,0.583940,0.583492
3,0.000000,0.000000,0.000000,-0.089929,-0.002183,-0.025430,0.073306,0.051638,-0.045054,-0.063927,...,0.574118,0.575576,0.577022,0.572972,0.575872,0.578356,0.579722,0.582381,0.583287,0.582840
4,0.000000,0.000000,0.000000,0.000000,0.093743,0.030967,0.126950,0.100717,-0.015129,-0.029951,...,0.574305,0.575773,0.577199,0.573147,0.576047,0.578529,0.579898,0.582554,0.583454,0.583007
5,0.000000,0.000000,0.000000,0.000000,0.000000,-0.030097,0.113839,0.075132,-0.051647,-0.072763,...,0.571586,0.573051,0.574468,0.570444,0.573348,0.575824,0.577184,0.579827,0.580708,0.580264
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145778,0.112019,-0.045798,-0.070389,...,0.572432,0.573894,0.575327,0.571313,0.574227,0.576712,0.578066,0.580722,0.581602,0.581150
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.077468,-0.247839,-0.241351,...,0.568986,0.570446,0.571842,0.567843,0.570748,0.573218,0.574563,0.577199,0.578047,0.577598
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.178035,-0.222024,...,0.569187,0.570644,0.572044,0.568042,0.570940,0.573418,0.574765,0.577407,0.578265,0.577824
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.034482,...,0.570331,0.571783,0.573179,0.569191,0.572088,0.574552,0.575888,0.578537,0.579384,0.578928
